# AgentCore Gateway を介した AgentCore Runtime エージェントと Elasticsearch の接続

## 概要

組織のデータストアからインテリジェントに情報を検索・取得できる AI エージェントの構築は、強力なカスタマーサポートとナレッジ管理ソリューションを作成する上で不可欠です。しかし、セキュリティ、スケーラビリティ、標準化を維持しながら、AI エージェントを Elasticsearch などのエンタープライズ検索システムに接続することは複雑になる可能性があります。

Amazon Bedrock AgentCore Gateway は、Elasticsearch などのサードパーティのベクトルストアを統合するのに役立ち、既存のデータインフラストラクチャで柔軟に作業できます。Elasticsearch は、強力なベクトル検索機能と従来の全文検索、高度なフィルタリング、実証済みのスケーラビリティを組み合わせているため、エンタープライズ AI アプリケーションに特に適しています。これにより、多様な大規模データセットを検索する必要がある組織に最適です。

このチュートリアルでは、Amazon Bedrock AgentCore Gateway を介して Elasticsearch データに対してセマンティック検索を実行する AI エージェントを構築する方法をデモンストレーションします。AWS Lambda 関数を Model Context Protocol（MCP）準拠のツールに変換し、AgentCore Runtime で実行されている AI エージェントに接続する方法を学びます。これらすべてを、デュアル認証によるエンタープライズグレードのセキュリティを維持しながら行います。

### チュートリアル詳細

| 情報                  | 詳細                                                      |
|:---------------------|:----------------------------------------------------------|
| チュートリアルタイプ   | 中級 / 統合                                               |
| エージェントタイプ     | カスタマーサポート RAG エージェント                        |
| AgentCore コンポーネント | Gateway、Identity、Runtime                               |
| エージェントフレームワーク | Strands Agents                                          |
| Gateway ターゲットタイプ | AWS Lambda                                              |
| データストア           | Elasticsearch                                            |
| Inbound 認証          | Amazon Cognito（OAuth）                                   |
| Outbound 認証         | AWS IAM                                                   |
| LLM モデル            | Anthropic Claude Haiku 4.5                               |
| 例の複雑さ            | 中級                                                      |
| 使用 SDK              | boto3、bedrock-agentcore、bedrock-agentcore-starter-toolkit |

### 学習内容

このチュートリアルでは、以下を学びます：
1. AgentCore Gateway で使用するための AWS Lambda 関数のパッケージング方法
2. セキュアなアクセスのためのデュアル認証（インバウンド OAuth とアウトバウンド IAM）の設定方法
3. Gateway ツールと対話する Strands エージェントの構築方法
4. 完全なソリューションを AgentCore Runtime にデプロイする方法
5. Elasticsearch でのエンドツーエンド RAG 機能のテスト方法

### アーキテクチャ

このチュートリアルでは、Elasticsearch からポリシー情報を取得するカスタマーサポートエージェントをデモンストレーションします：

<div style="text-align:left">
    <img src="images/runtime-gateway-elastic.png" width="90%"/>
</div>

**アーキテクチャフロー：**
1. **ユーザーリクエスト**: クライアントが認証済みリクエストを AgentCore Runtime に送信
2. **Inbound 認証**: Cognito が Gateway アクセス用の OAuth トークンを検証
3. **エージェント処理**: Strands エージェントがどのツールを使用するかを決定
4. **Gateway 呼び出し**: エージェントが Gateway 経由で MCP ツールを呼び出し
5. **Outbound 認証**: Gateway が IAM を使用して Lambda を呼び出し
6. **データ取得**: Lambda が Elasticsearch にクエリを実行して結果を返す
7. **レスポンス生成**: エージェントがユーザー向けの最終レスポンスを合成

## 0. 前提条件

このチュートリアルを実行するには、以下が必要です：

**ソフトウェア要件：**
* Python 3.10 以降
* Jupyter notebook 環境

**AWS 要件：**
* 以下の適切な権限で設定された AWS 認証情報：
  - Amazon Bedrock（モデルアクセス）
  - AWS Lambda（関数の作成と呼び出し）
  - Amazon Cognito（ユーザープール管理）
  - Amazon ECR（コンテナレジストリ）
  - IAM（ロールとポリシー管理）
  - AWS Systems Manager Parameter Store
* Amazon Bedrock モデルアクセス（Claude Haiku 4.5）

**Elasticsearch 要件：**
* アクティブな Elasticsearch インデックス
* Elasticsearch 認証情報：
  - `ELASTIC_ENDPOINT_URL`: Elasticsearch エンドポイント
  - `ELASTIC_API_KEY`: 認証用 API キー
  - `ELASTIC_INDEX_NAME`: インデックス名

### Elasticsearch 設定のセットアップ

続行する前に、Elasticsearch 接続の詳細を設定してください：

In [ ]:
# これらの値を実際の Elasticsearch の詳細に置き換えてください
ELASTIC_ENDPOINT_URL = "Your Elasticsearch endpoint"
ELASTIC_API_KEY = "API key for authentication"
ELASTIC_INDEX_NAME = "Index name"

### 必要な依存関係のインストール

必要なすべての Python パッケージをインストールしましょう：

In [ ]:
!pip install -qUr requirements.txt

### 環境のセットアップ

必要なライブラリをインポートし、環境を設定しましょう：

In [ ]:
# 標準ライブラリのインポート
import os
import sys
import uuid
import time
import shutil
import zipfile
import subprocess
from pathlib import Path

# AWS SDK のインポート
import boto3
from botocore.exceptions import ClientError

# AWS リージョンを設定
REGION = os.getenv('AWS_REGION', 'us-east-1')

# ユーティリティのインポートを設定
current_dir = os.getcwd() if '__file__' not in globals() else os.path.dirname(os.path.abspath(__file__))
sys.path.insert(0, current_dir)

# チュートリアルユーティリティをインポート
import utils
from utils_execution import put_ssm_parameter

print(f"✅ リージョン {REGION} の環境を設定しました")

### Elasticsearch へのポリシードキュメントのアップロード

RAG エージェントを構築する前に、Elasticsearch インデックスに製品ポリシードキュメントを格納する必要があります。このステップでは、`policies/` ディレクトリのすべてのテキストファイルを Elasticsearch インスタンスにバルクアップロードします。

> ⏱️ **パフォーマンス**: バルクインデックスは大量のドキュメントセットに効率的です。このプロセスは通常、数十のドキュメントで数秒かかります。

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

print("📤 ポリシードキュメントを Elasticsearch にバルクアップロード中...\n")

# Elasticsearch クライアントを初期化
try:
    es_client = Elasticsearch(
        ELASTIC_ENDPOINT_URL,
        api_key=ELASTIC_API_KEY
    )
    
    # 接続をテスト
    if es_client.ping():
        print("✅ Elasticsearch に接続しました")
    else:
        print("❌ Elasticsearch への接続に失敗しました")
        raise Exception("Elasticsearch 接続失敗")
        
except Exception as e:
    print(f"❌ Elasticsearch 接続エラー: {e}")
    raise

# ポリシーディレクトリを定義
policies_dir = Path("policies")

if not policies_dir.exists():
    print(f"❌ ディレクトリが見つかりません: {policies_dir}")
    print("💡 'policies/' ディレクトリを作成し、.txt ポリシーファイルを追加してください。")
    raise FileNotFoundError(f"ディレクトリ {policies_dir} が存在しません")

# すべての .txt ファイルを取得
policy_files = list(policies_dir.glob("*.txt"))

if not policy_files:
    print(f"⚠️  {policies_dir} に .txt ファイルが見つかりません")
    print("💡 policies/ ディレクトリにポリシードキュメントを .txt ファイルとして追加してください。")
else:
    print(f"📁 {len(policy_files)} 件のポリシードキュメントが見つかりました\n")

# バルクインデックス用にドキュメントを準備
def generate_docs():
    """バルクインデックス用のドキュメントを生成するジェネレータ関数"""
    for idx, file_path in enumerate(policy_files, 1):
        try:
            # ファイルの内容を読み取り
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            # ドキュメント構造を作成
            doc = {
                "_index": ELASTIC_INDEX_NAME,
                "_id": file_path.stem,  # ファイル名（拡張子なし）をドキュメント ID として使用
                "_source": {
                    "filename": file_path.name,
                    "attachment": {
                        "content": content
                    },
                    "document_type": "policy",
                    "indexed_at": "2024-01-01T00:00:00Z"  # datetime.now().isoformat() を使用することも可能
                }
            }
            
            print(f"   [{idx}/{len(policy_files)}] 準備中: {file_path.name}")
            yield doc
            
        except Exception as e:
            print(f"   ⚠️  {file_path.name} の読み取りエラー: {e}")
            continue

# バルクインデックスを実行
try:
    print("\n⏳ ドキュメントをインデックス中...\n")
    
    # バルク操作を実行
    success, failed = bulk(
        es_client,
        generate_docs(),
        raise_on_error=False,
        refresh=True  # ドキュメントをすぐに検索可能にする
    )
    
    print("\n" + "=" * 70)
    print(f"✅ 正常にインデックス完了: {success} 件のドキュメント")
    
    if failed:
        print(f"⚠️  インデックス失敗: {failed} 件のドキュメント")
    
    # インデックスを検証
    count_response = es_client.count(index=ELASTIC_INDEX_NAME)
    total_docs = count_response['count']
    print(f"📊 インデックス '{ELASTIC_INDEX_NAME}' 内の総ドキュメント数: {total_docs}")
    print("=" * 70)
    
    # サンプルドキュメント構造を表示
    if total_docs > 0:
        print("\n📄 サンプルドキュメント構造:")
        sample_doc = es_client.search(
            index=ELASTIC_INDEX_NAME,
            size=1
        )['hits']['hits'][0]
        
        print(f"   ドキュメント ID: {sample_doc['_id']}")
        print(f"   ファイル名: {sample_doc['_source'].get('filename')}")
        print(f"   コンテンツプレビュー: {sample_doc['_source']['attachment']['content'][:100]}...")
    
    print("\n🎉 バルクアップロードが正常に完了しました！\n")
    
except Exception as e:
    print(f"\n❌ バルクインデックス中のエラー: {e}")
    import traceback
    traceback.print_exc()
    raise

finally:
    # Elasticsearch 接続を閉じる
    es_client.close()
    print("✅ Elasticsearch 接続を閉じました")

## 1. Elasticsearch 統合用の Lambda 関数の作成

最初のステップは、Elasticsearch にクエリを実行する AWS Lambda 関数を作成することです。この Lambda 関数は、後で AgentCore Gateway によって MCP ツールに変換され、AI エージェントからアクセス可能になります。

### Lambda 関数の設計

Lambda 関数は以下を行います：
1. 検索クエリを入力として受け入れる
2. 保存された認証情報を使用して Elasticsearch に接続
3. ドキュメントコンテンツ全体でマルチマッチクエリを実行
4. 関連する検索結果をエージェントに返す

### ステップ 1: プロジェクト構造の作成

まず、Lambda 関数コードを整理するためのディレクトリを作成します：

In [ ]:
# Lambda パッケージ用のディレクトリ構造を作成
os.makedirs("elastic_lambda_code/package", exist_ok=True)
print("✅ Lambda プロジェクトディレクトリを作成しました")

### ステップ 2: Lambda 依存関係の定義

Lambda 関数には、データストアにクエリを実行するために Elasticsearch Python クライアントが必要です：

In [ ]:
%%writefile elastic_lambda_code/requirements.txt
elasticsearch

### ステップ 3: Lambda 関数コードの作成

実際の Lambda 関数を作成しましょう。この関数は AgentCore Gateway によって呼び出され、エージェントから渡されたツール名とパラメータに基づいて Elasticsearch にクエリを実行します。

#### 主要コンポーネント：

1. **環境変数**: 関数は環境変数から Elasticsearch 認証情報を読み取ります（Lambda 作成時に設定）
2. **イベントスキーマ**: `event` パラメータにはツール入力（例：検索クエリ）が含まれます
3. **コンテキストメタデータ**: `context.client_context.custom` には、ツール名を含む Gateway メタデータが含まれます
4. **ツール名の解析**: Gateway ターゲットプレフィックスを削除して実際のツール名を抽出します
5. **Elasticsearch クエリ**: ドキュメントコンテンツ全体でマルチマッチ検索を実行します

> 💡 **Gateway コンテキスト**: AgentCore Gateway は、Gateway ID、ターゲット ID、ツール名を含むメタデータを Lambda コンテキストに自動的に注入します。これにより、単一の Lambda 関数で複数のツールを実装できます。

In [ ]:
%%writefile elastic_lambda_code/lambda_function_code.py
import json
import boto3
from elasticsearch import Elasticsearch
import os

# 環境変数から Elasticsearch 設定を読み込み
endpoint_url = os.environ['ELASTIC_ENDPOINT_URL_ENV']
api_key = os.environ['ELASTIC_API_KEY_ENV']
index_name = os.environ['ELASTIC_INDEX_NAME_ENV']

# Elasticsearch クライアントを初期化
client = Elasticsearch(
    endpoint_url,
    api_key=api_key
)

def lambda_handler(event, context):
    """
    AgentCore Gateway 経由の Elasticsearch 検索用 Lambda ハンドラー
    
    Args:
        event: Gateway で定義された inputSchema に一致するツール入力を含む
        context: context.client_context.custom に Gateway メタデータを含む
    
    コンテキスト構造:
        {
            'bedrockAgentCoreGatewayId': 'Y02ERAYBHB',
            'bedrockAgentCoreTargetId': 'RQHDN3J002',
            'bedrockAgentCoreMessageVersion': '1.0',
            'bedrockAgentCoreToolName': 'target_name___elastic_rag_tool',
            'bedrockAgentCoreSessionId': ''
        }
    """
    
    # Gateway コンテキストからツール名を抽出
    # Gateway はツール名にターゲット名とデリミタをプレフィックスとして付加
    delimiter = "___"
    original_tool_name = context.client_context.custom['bedrockAgentCoreToolName']
    tool_name = original_tool_name[original_tool_name.index(delimiter) + len(delimiter):]
    
    # 適切なツールハンドラーにルーティング
    if tool_name == 'elastic_rag_tool':
        # イベントから検索クエリを抽出
        query = event["query"]
        
        # Elasticsearch クエリを構築
        # ドキュメントコンテンツフィールド全体を検索するために multi_match を使用
        retriever_object = {
            "standard": {
                "query": {
                    "multi_match": {
                        "query": query,
                        "fields": [
                            "attachment.content" 
                        ]
                    }
                }
            }
        }
        
        # 検索を実行
        search_response = client.search(
            index=index_name,
            retriever=retriever_object,
        )
        
        # 結果を返す
        return {'statusCode': 200, 'body': search_response.body}
    else:
        # サポートされていないツールを処理
        return {'statusCode': 400, 'body': "Unsupported tool."}

### ステップ 4: Lambda デプロイメントバンドルのパッケージング

以下の手順でデプロイメントパッケージを作成します：
1. `package/` ディレクトリに依存関係をインストール
2. すべての依存関係を含む zip ファイルを作成
3. Lambda 関数コードを zip に追加

このプロセスは AWS Lambda の[デプロイメントパッケージ仕様](https://docs.aws.amazon.com/lambda/latest/dg/python-package.html)に従います。

In [ ]:
# 元のディレクトリを保存
original_dir = os.getcwd()
project_dir = "elastic_lambda_code"

try:
    print("📦 Lambda デプロイメントパッケージをビルド中...")
    
    # プロジェクトディレクトリに移動
    os.chdir(project_dir)
    print(f"✅ 移動先: {os.getcwd()}")
    
    # パッケージディレクトリが存在することを確認
    os.makedirs("package", exist_ok=True)
    print("✅ パッケージディレクトリを作成しました")
    
    # 依存関係をインストール
    print("📥 依存関係をインストール中...")
    subprocess.check_call([
        sys.executable, "-m", "pip", "install", 
        "-r", "requirements.txt",
        "--target", "./package",
        "--quiet"
    ])
    print("✅ 依存関係を ./package にインストールしました")
    
    # 依存関係を含む zip を作成
    os.chdir("package")
    print("🗜️ デプロイメントパッケージを作成中...")
    subprocess.check_call([
        "zip", "-r", "../my_deployment_package.zip", ".", "-q"
    ])
    print("✅ 依存関係を含む zip を作成しました")
    
    # Lambda 関数コードを追加
    os.chdir("..")
    subprocess.check_call([
        "zip", "my_deployment_package.zip", "lambda_function_code.py", "-q"
    ])
    print("✅ lambda_function_code.py を zip に追加しました")
    
    # 親ディレクトリに移動
    shutil.move("my_deployment_package.zip", "../my_deployment_package.zip")
    print("✅ デプロイメントパッケージをプロジェクトルートに移動しました")
    
finally:
    # 常に元のディレクトリに戻る
    os.chdir(original_dir)
    print(f"✅ 戻り先: {os.getcwd()}")

print("\n🎉 Lambda デプロイメントパッケージが正常に作成されました！")

### ステップ 5: Lambda 関数のデプロイ

デプロイメントパッケージを使用して AWS に実際の Lambda 関数を作成しましょう。ユーティリティ関数は以下を行います：
- Lambda 用の IAM 実行ロールを作成
- デプロイメントパッケージをアップロード
- Elasticsearch アクセス用の環境変数を設定
- 適切なタイムアウトとメモリ設定を構成

> ⚠️ **セキュリティに関する注意**: Lambda 関数は、環境変数に保存された API キーを介して Elasticsearch インスタンスにアクセスできます。本番環境では、セキュリティを強化するために AWS Secrets Manager の使用を検討してください。

In [ ]:
print("🚀 Lambda 関数を作成中...")

# Lambda 環境変数を設定
environment_variables = {
    'ELASTIC_ENDPOINT_URL_ENV': ELASTIC_ENDPOINT_URL,
    'ELASTIC_API_KEY_ENV': ELASTIC_API_KEY,
    'ELASTIC_INDEX_NAME_ENV': ELASTIC_INDEX_NAME,
}

# Lambda 関数を作成（最大2分かかる場合があります）
lambda_resp = utils.create_gateway_lambda("my_deployment_package.zip", environment_variables)

# 作成ステータスを確認
if lambda_resp is not None:
    if lambda_resp['exit_code'] == 0:
        print(f"✅ Lambda 関数が正常に作成されました")
        print(f"   ARN: {lambda_resp['lambda_function_arn']}")
        lambda_function_arn = lambda_resp['lambda_function_arn']
    else:
        print(f"❌ Lambda 関数の作成に失敗しました: {lambda_resp['lambda_function_arn']}")
        raise Exception("Lambda 作成失敗")
else:
    print("❌ Lambda 関数の作成が None を返しました")
    raise Exception("Lambda 作成失敗")

## 2. AgentCore Gateway の認証設定

本番 AI システムを構築する際、セキュリティは最も重要です。AgentCore Gateway は、多層防御を提供する**デュアル認証モデル**を実装しています：

### デュアル認証の理解

**1. インバウンド認証（誰が Gateway を呼び出せるか？）**
- Gateway ツールにアクセスしようとするユーザー/エージェントを検証
- Amazon Cognito からの OAuth 2.0 トークンを使用
- 認可されたクライアントのみが MCP ツールを呼び出せることを保証

**2. アウトバウンド認証（Gateway はどのようにリソースにアクセスするか？）**
- バックエンドサービス（Lambda、API）を呼び出すための Gateway の認可
- Lambda 呼び出しに AWS IAM ロールを使用
- ダウンストリームリソースへのセキュアで監査可能なアクセスを有効化

### 認証フロー

```
クライアント（エージェント）
    │
    │ [1] リクエスト + OAuth トークン
    ↓
AgentCore Gateway
    │
    │ [2] OAuth トークンの検証（インバウンド認証）
    │
    │ [3] IAM ロールの引き受け（アウトバウンド認証）
    ↓
AWS Lambda → Elasticsearch
```

両方の認証レイヤーをセットアップしましょう。

## 3. インバウンド認可のための Amazon Cognito のセットアップ

Amazon Cognito は、ユーザー認証を管理し、Gateway へのアクセスを許可する OAuth トークンを発行するアイデンティティプロバイダー（IdP）として機能します。

### 作成するもの

1. **Cognito ユーザープール**: ユーザー ID と認証設定のコンテナ
2. **リソースサーバー**: きめ細かなアクセス制御のためのカスタム OAuth スコープを定義
3. **アプリクライアント**: プログラムによるアクセス用のマシン間（M2M）クライアント
4. **OAuth スコープ**: `gateway:read` と `gateway:write` 権限

> 💡 **本番環境のヒント**: 本番環境では、最小権限の原則を使用してください。各クライアントの機能に必要な最小限のスコープのみを付与します。

In [ ]:
print("🔐 Amazon Cognito 認証をセットアップ中...\n")

# 設定
USER_POOL_NAME = "agentcore-gateway-elasticsearch-pool"
RESOURCE_SERVER_ID = "elasticsearch-gateway"
RESOURCE_SERVER_NAME = "Elasticsearch Gateway Resource Server"
CLIENT_NAME = "elasticsearch-gateway-client"

# アクセス制御用の OAuth スコープを定義
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Gateway ツールへの読み取りアクセス"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Gateway ツールへの書き込みアクセス"}
]
scope_string = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

# Cognito クライアントを初期化
cognito = boto3.client("cognito-idp", region_name=REGION)

# ステップ 1: ユーザープールを作成または取得
print("📋 ユーザープールを作成/取得中...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"✅ ユーザープール ID: {user_pool_id}")

# ステップ 2: リソースサーバーを作成または取得
print("\n🔧 リソースサーバーを設定中...")
utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("✅ リソースサーバーを OAuth スコープで設定しました")

# ステップ 3: M2M アプリクライアントを作成または取得
print("\n🔑 マシン間クライアントを作成中...")
client_id, client_secret = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"✅ クライアント ID: {client_id}")
print("✅ クライアントシークレットが生成されました（安全に保存済み）")

# ステップ 4: Gateway 設定用の discovery URL を生成
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(f"\n🌐 Discovery URL: {cognito_discovery_url}")

print("\n🎉 Cognito 認証のセットアップが完了しました！")

## 4. AgentCore Gateway の作成

AgentCore Gateway を作成する準備ができました。これは、Lambda 関数を AI エージェントからアクセス可能な MCP 準拠のツールに変換する中央ハブです。

### AgentCore Gateway とは？

AgentCore Gateway は、以下を提供するフルマネージドサービスです：
- **統合の標準化**: 異なるバックエンドサービス間で統一された MCP インターフェースを提供
- **セキュリティの管理**: インバウンドとアウトバウンドの両方の認証を処理
- **インフラストラクチャの排除**: 管理や保守するサーバーが不要
- **自動スケーリング**: 設定なしで変動する負荷に対応

### Gateway コンポーネント

Gateway の作成には以下が関係します：
1. **IAM 実行ロール**: Lambda 関数を呼び出す権限を Gateway に付与
2. **Gateway リソース**: 認証設定を持つメイン Gateway インスタンス
3. **Gateway ターゲット**: Gateway を Lambda 関数に接続

各コンポーネントを作成しましょう。

### ステップ 1: IAM 実行ロールの作成

Gateway は、代わりに Lambda 関数を呼び出すための IAM ロールが必要です。このロールは、先ほど説明した**アウトバウンド認証**を実装します。

In [ ]:
print("🔐 Gateway 用の IAM 実行ロールを作成中...")

# 適切な信頼ポリシーと権限を持つロールを作成
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("elasticsearch-gateway")
gateway_role_arn = agentcore_gateway_iam_role['Role']['Arn']

print(f"✅ Gateway IAM ロールを作成しました")
print(f"   ARN: {gateway_role_arn}")

### ステップ 2: Cognito 認可を持つ Gateway の作成

Gateway 自体を作成し、インバウンド認証に Cognito ユーザープールを使用するように設定します。

#### 設定の詳細：

- **プロトコルタイプ**: `MCP` - 標準化されたツールインターフェース用の Model Context Protocol
- **認可タイプ**: `CUSTOM_JWT` - 認証に Cognito JWT トークンを使用
- **許可されたクライアント**: Gateway へのアクセスが許可された Cognito クライアント ID のリスト
- **Discovery URL**: トークン検証のための Cognito の OIDC ディスカバリエンドポイント

> 💡 **セキュリティに関する注意**: `allowedClients` にリストされているクライアントのみが、有効な JWT トークンを持っていても Gateway ツールを正常に呼び出すことができます。これにより、追加のアクセス制御レイヤーが提供されます。

In [ ]:
print("🚪 AgentCore Gateway を作成中...\n")

# Gateway クライアントを初期化
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)

# Cognito JWT 認可を設定
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # Cognito クライアント ID と一致する必要があります
        "discoveryUrl": cognito_discovery_url
    }
}

# 一意の Gateway 名を生成
gateway_name = f'ElasticsearchGateway-{str(uuid.uuid4())[:8]}'

# Gateway を作成
try:
    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn=gateway_role_arn,
        protocolType='MCP',
        authorizerType='CUSTOM_JWT',
        authorizerConfiguration=auth_config,
        description='Lambda 経由の Elasticsearch 統合用 AgentCore Gateway'
    )
    
    # Gateway の詳細を抽出
    gateway_id = create_response["gatewayId"]
    gateway_url = create_response["gatewayUrl"]
    
    print(f"✅ Gateway が正常に作成されました！")
    print(f"   Gateway ID: {gateway_id}")
    print(f"   Gateway URL: {gateway_url}")
    
    # 後で使用するために Gateway ID を Parameter Store に保存
    put_ssm_parameter("/app/customer_support_elastic/agentcore/gateway_id", gateway_id)
    print(f"\n✅ Gateway ID を SSM Parameter Store に保存しました")
    
except ClientError as e:
    print(f"❌ Gateway 作成エラー: {e}")
    raise

## 5. Gateway ターゲットと MCP ツールの作成

Gateway のセットアップの最後のステップは、**Gateway ターゲット**を作成することです。これは Lambda 関数を Gateway に接続し、MCP ツールとしてどのように公開するかを定義します。

### Gateway ターゲットの理解

Gateway ターゲットは以下を行います：
- **Lambda を MCP にマッピング**: Lambda 関数を 1 つ以上の MCP ツールに変換
- **ツールスキーマを定義**: ツール名、説明、入力パラメータを指定
- **認証情報を設定**: Gateway が Lambda に認証する方法を設定（IAM、API キー、OAuth）

### ツール定義

`elastic_rag_tool` という単一のツールを作成します：
- **名前**: `elastic_rag_tool`
- **目的**: Elasticsearch でセマンティック検索を実行
- **入力**: `query` 文字列パラメータ
- **出力**: Elasticsearch からの関連検索結果

### ツールスキーマ構造

ツールスキーマは JSON Schema 形式に従います：
```json
{
    "name": "elastic_rag_tool",
    "description": "tool to perform search on Elastic",
    "inputSchema": {
        "type": "object",
        "properties": {
            "query": {"type": "string"}
        },
        "required": ["query"]
    }
}
```

このスキーマは、ツールが期待するパラメータと必須のものを AI エージェントに伝えます。

> 💡 **マルチツール Lambda**: 単一の Lambda 関数で、コンテキスト内のツール名をチェックすることで複数のツールを実装できます（Lambda コードで行ったように）。

In [ ]:
print("🔧 Lambda 用の Gateway ターゲットを作成中...\n")

# MCP ツールスキーマを持つ Lambda ターゲット設定を定義
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_function_arn,
            "toolSchema": {
                "inlinePayload": [
                    {
                        "name": "elastic_rag_tool",
                        "description": "Elasticsearch で関連する製品ポリシー情報を検索します。顧客が製品ドキュメント、トラブルシューティングガイド、保証情報、またはその他の製品関連の質問をした場合に、このツールを使用します。",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "query": {
                                    "type": "string",
                                    "description": "Elasticsearch で関連ドキュメントを検索するための検索クエリ"
                                }
                            },
                            "required": ["query"]
                        }
                    }
                ]
            }
        }
    }
}

# 認証情報プロバイダーを設定（アウトバウンド認証）
credential_config = [
    {
        "credentialProviderType": "GATEWAY_IAM_ROLE"  # Lambda 呼び出しに Gateway の IAM ロールを使用
    }
]

# 一意のターゲット名を生成
target_name = f'ElasticSearchTarget-{str(uuid.uuid4())[:8]}'

# Gateway ターゲットを作成
try:
    response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name=target_name,
        description='Elasticsearch RAG 機能用の Lambda ターゲット',
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=credential_config
    )
    
    target_id = response['targetId']
    
    print(f"✅ Gateway ターゲットが正常に作成されました！")
    print(f"   ターゲット ID: {target_id}")
    print(f"   ターゲット名: {target_name}")
    print(f"\n🎉 Lambda 関数が MCP ツールとして利用可能になりました: {target_name}___elastic_rag_tool")
    
except ClientError as e:
    print(f"❌ Gateway ターゲット作成エラー: {e}")
    raise

## 6. Strands エージェントによる Gateway のテスト

完全なインフラストラクチャをセットアップしたので、Gateway を介して Elasticsearch データにアクセスできる Strands エージェントを作成してテストしましょう。

### MCP 統合の仕組み

Strands エージェントは、Model Context Protocol（MCP）仕様を使用して AgentCore Gateway と統合します。フローは以下の通りです：

```
┌─────────────────┐
│  Strands エージェント  │
└────────┬────────┘
         │
         │ 1. ListTools（利用可能なツールの検出）
         ↓
┌─────────────────────┐
│  AgentCore Gateway  │ ← OAuth トークン
└─────────┬───────────┘
          │
          │ 2. InvokeTool（特定のツールの呼び出し）
          ↓
┌──────────────────┐
│   AWS Lambda     │
└────────┬─────────┘
         │
         │ 3. クエリ
         ↓
┌──────────────────┐
│  Elasticsearch   │
└──────────────────┘
```

### MCP クライアント設定

Strands エージェントは `MCPClient` を使用して Gateway と通信します：
- **トランスポート**: Bearer トークン認証を使用した HTTP(S)
- **ヘッダー**: インバウンド認証用の OAuth トークン
- **プロトコル**: 標準 MCP 操作（ListTools、InvokeTools）

まず、Cognito からアクセストークンをリクエストしましょう。

### ステップ 1: OAuth アクセストークンのリクエスト

Gateway を呼び出す前に、Cognito から OAuth アクセストークンを取得する必要があります。このトークンは、ID を証明し、Gateway ツールへのアクセスを許可します。

> ⏰ **トークンの有効期限**: Cognito アクセストークンは通常 1 時間有効です。トークンの有効期限が切れた場合は、同じプロセスを使用して新しいトークンをリクエストする必要があります。

In [ ]:
print("🔑 Cognito から OAuth アクセストークンをリクエスト中...\n")

# Cognito ドメインの伝播を待機
print("⏳ Cognito ドメインの伝播を待機中...")
time.sleep(10)

try:
    # アクセストークンをリクエスト
    token_response = utils.get_token(user_pool_id, client_id, client_secret, scope_string, REGION)
    access_token = token_response["access_token"]
    
    print("✅ アクセストークンを正常に取得しました")
    print(f"   トークンタイプ: Bearer")
    print(f"   スコープ: {scope_string}")
    print(f"   トークン（省略）: {access_token[:50]}...")
    
except Exception as e:
    print(f"❌ トークン取得エラー: {e}")
    print("\n💡 ヒント: 失敗した場合は、Cognito ドメインの伝播が完了するまで数分待ってから再試行してください。")
    raise

### ステップ 2: MCP クライアントとエージェントの作成

MCP クライアント統合を持つ Strands エージェントを作成しましょう。エージェントは以下を行います：
1. OAuth トークンを使用して Gateway に接続
2. `ListTools` MCP 操作で利用可能なツールを検出
3. それらのツールを設定にロード
4. ユーザークエリに基づいて呼び出す準備完了

> 💡 **モデル選択**: Claude Haiku 4.5 を使用していますが、Bedrock がサポートする任意のモデルに置き換えることができます。

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent
import logging

# エージェントの推論を確認するためにロギングを設定
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)
logging.getLogger("strands").setLevel(logging.INFO)

print("🤖 MCP ツールを持つ Strands エージェントを作成中...\n")

# MCP クライアント用の HTTP トランスポートファクトリを作成
def create_streamable_http_transport():
    """MCP 用の認証済み HTTP トランスポートを作成するファクトリ関数"""
    return streamablehttp_client(
        gateway_url,
        headers={"Authorization": f"Bearer {access_token}"}
    )

# MCP クライアントを初期化
mcp_client = MCPClient(create_streamable_http_transport)

# Bedrock モデルを作成
model = BedrockModel(
    model_id="us.anthropic.claude-haiku-4-5-20251001-v1:0",
    temperature=0.3,
    region_name=REGION,
)

print("✅ MCP クライアントを設定しました")
print("✅ Bedrock モデルを初期化しました")

### ステップ 3: エージェントのテスト

完全な統合を検証するために、様々なクエリでエージェントをテストしましょう：

1. **利用可能なツールの一覧** - MCP ツール検出の検証
2. **Elasticsearch 検索** - RAG 機能のテスト
3. **顧客クエリへの回答** - エンドツーエンドワークフローの検証

In [ ]:
print("🧪 MCP ツールでエージェントをテスト中...\n")
print("=" * 70)

with mcp_client:
    # ステップ 1: 利用可能なツールを検出
    print("\n📋 ステップ 1: MCP ツールを検出中...")
    tools = mcp_client.list_tools_sync()
    print(f"✅ {len(tools)} 件のツールが見つかりました")
    
    # ステップ 2: ツールを持つエージェントを作成
    print("\n🤖 ステップ 2: エージェントを作成中...")
    agent = Agent(model=model, tools=tools)
    print(f"✅ エージェントをツールで作成しました: {agent.tool_names}")
    
    # ステップ 3: ツール一覧クエリをテスト
    print("\n" + "=" * 70)
    print("テスト 1: 利用可能なツールの一覧表示")
    print("=" * 70)
    response = agent("Hi, can you list all tools available to you?")
    print(f"\n🤖 エージェント: {response.message['content'][0]['text']}")
    
    # ステップ 4: Elasticsearch 検索をテスト
    print("\n" + "=" * 70)
    print("テスト 2: Elasticsearch で製品情報を検索")
    print("=" * 70)
    response = agent("Use the elastic_rag_tool to find information about how to troubleshoot the LAPTOP ULTRA 15.6")
    print(f"\n🤖 エージェント: {response.message['content'][0]['text']}")

print("\n" + "=" * 70)
print("✅ すべてのテストが正常に完了しました！")
print("=" * 70)

## 7. AgentCore Runtime へのデプロイ

これまで、このノートブックでエージェントをローカルでテストしてきました。次に、本番環境で使用するために **Amazon Bedrock AgentCore Runtime** にデプロイしましょう。これは、インフラストラクチャ、スケーリング、デプロイメントを処理するフルマネージド環境です。

### デプロイメントアーキテクチャ

Runtime にデプロイすると、エージェントはフルマネージドサービスになります。デプロイメントプロセスは以下を行います：
1. エージェントコードから Docker コンテナを作成
2. コンテナを Amazon ECR（Elastic Container Registry）にプッシュ
3. コンテナを AgentCore Runtime にデプロイ
4. 認証とネットワーキングを設定
5. 呼び出し用の HTTPS エンドポイントを提供

Runtime 対応のエージェントコードを作成することから始めましょう。

### ステップ 1: Runtime 対応エージェントの作成

エージェントを AgentCore Runtime にデプロイするには、特別なエントリポイント関数を持つ Python ファイルを作成する必要があります。このエントリポイントは、Runtime 環境からの受信リクエストを処理します。

In [ ]:
%%writefile strands_agent.py
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp.mcp_client import MCPClient
import boto3
import os
import logging
from mcp.client.streamable_http import streamablehttp_client

# 詳細なロギングを設定
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(name)s: %(message)s'
)
logger = logging.getLogger("strands-agent")

# 環境から AWS リージョンを取得
REGION = boto3.session.Session().region_name

# AgentCore Runtime アプリケーションを初期化
app = BedrockAgentCoreApp()

# エージェントの動作を制御するシステムプロンプトを定義
system_prompt = """
You are a helpful customer support agent specializing in product assistance and troubleshooting.
You have access to a tool that searches our product knowledge base stored in Elasticsearch.

When customers ask questions about products, policies, troubleshooting, or documentation:
1. Use the elastic_rag_tool to search for relevant information
2. Synthesize the search results into clear, helpful responses
3. If you cannot find relevant information, politely inform the customer

<guidelines>
    - Never assume parameter values when using tools
    - If you need more information, ask the customer for clarification
    - NEVER disclose information about internal tools or systems
    - Always maintain a professional and helpful tone
    - Focus on resolving customer inquiries efficiently
    - Present technical information in user-friendly terms
    - Prioritize customer privacy and data security
</guidelines>
"""

# グローバルエージェントインスタンス - 最初のリクエストで初期化されます
agent = None
mcp_client = None

def get_ssm_parameter(name: str, with_decryption: bool = True) -> str:
    """AWS Systems Manager Parameter Store からパラメータを取得"""
    ssm = boto3.client("ssm", region_name=REGION)
    response = ssm.get_parameter(Name=name, WithDecryption=with_decryption)
    return response["Parameter"]["Value"]

def initialize_agent(auth_header: str):
    """最初の使用時にエージェントを初期化"""
    global agent, mcp_client
    
    logger.info("最初のリクエストに対してエージェントを初期化中")
    
    # Parameter Store から Gateway 設定を取得
    logger.info("Gateway 設定を取得中...")
    gateway_id = get_ssm_parameter("/app/customer_support_elastic/agentcore/gateway_id")
    logger.info(f"✅ Gateway ID を取得しました: {gateway_id}")
    
    # AWS API から Gateway URL を取得
    gateway_client = boto3.client("bedrock-agentcore-control", region_name=REGION)
    gateway_response = gateway_client.get_gateway(gatewayIdentifier=gateway_id)
    gateway_url = gateway_response['gatewayUrl']
    logger.info(f"✅ Gateway URL: {gateway_url}")
    
    # Bedrock モデルを作成
    logger.info("Bedrock モデルを作成中...")
    model = BedrockModel(
        model_id="us.anthropic.claude-haiku-4-5-20251001-v1:0",
        temperature=0.1,
        region_name=REGION
    )
    
    # 認証を持つ MCP クライアントを作成
    logger.info("MCP クライアントを作成中...")
    mcp_client = MCPClient(lambda: streamablehttp_client(
        url=gateway_url,
        headers={"Authorization": auth_header}
    ))
    
    # MCP クライアントを初期化してツールを取得
    mcp_client.__enter__()
    tools = mcp_client.list_tools_sync()
    logger.info(f"✅ Gateway から {len(tools)} 件のツールをロードしました")
    
    # ツールを持つエージェントを作成
    logger.info("ツールを持つエージェントを作成中...")
    agent = Agent(
        model=model,
        tools=tools,
        system_prompt=system_prompt,
    )
    logger.info("✅ エージェントの初期化が正常に完了しました")

@app.entrypoint
async def invoke(payload, context=None):
    """
    AgentCore Runtime エントリポイント関数
    
    Args:
        payload: ユーザー入力を含む 'prompt' キーを持つ辞書
        context: Authorization トークンを含む request_headers を持つ Runtime コンテキスト
    
    Returns:
        str: エージェントのレスポンステキスト
    """
    global agent
    
    # ペイロードとコンテキストをログ出力
    logger.info(f"受信したペイロード: {payload}")
    logger.info(f"コンテキスト: {context}")
    
    # ユーザー入力を抽出
    user_input = payload.get("prompt")
    
    # 必須フィールドを検証
    if user_input is None:
        error_msg = "❌ エラー: ペイロードに 'prompt' フィールドがありません"
        logger.error(error_msg)
        return error_msg
    
    # リクエストヘッダーを取得（None の場合を処理）
    request_headers = context.request_headers or {}
    
    # Gateway 認証用の JWT トークンを抽出
    auth_header = request_headers.get('Authorization', '')
    
    if not auth_header:
        return "エラー: Authorization ヘッダーがありません。有効な OAuth トークンを提供してください。"
    
    logger.info(f"✅ Authorization ヘッダーが存在します: {auth_header[:20]}...")
    
    try:
        # 最初のリクエストでエージェントを初期化
        if agent is None:
            logger.info("最初のリクエスト - エージェントを初期化中")
            initialize_agent(auth_header)
        else:
            logger.info("既存のエージェントインスタンスを使用")
        
        # ユーザー入力でエージェントを呼び出し
        logger.info(f"🤖 クエリを処理中: {user_input[:50]}...")
        response = agent(user_input)
        
        # レスポンステキストを抽出して返す
        response_text = response.message["content"][0]["text"]
        logger.info(f"✅ エージェントのレスポンス: {response_text[:50]}...")
        
        return response_text
        
    except Exception as e:
        error_msg = f"リクエスト処理エラー: {str(e)}"
        logger.error(f"❌ {error_msg}", exc_info=True)
        return error_msg

if __name__ == "__main__":
    # HTTP サーバーを開始
    logger.info("AgentCore アプリケーションを起動中")
    app.run()

### ステップ 2: Runtime デプロイメントの設定

**AgentCore Starter Toolkit** を使用してデプロイメントを設定しましょう。

> 💡 **ヘッダー許可リスト**: スタック全体で OAuth トークンの伝播を有効にするために、`Authorization` ヘッダーをエージェントコードに転送するように Runtime を設定します。

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from utils_execution import create_agentcore_runtime_execution_role

print("⚙️ AgentCore Runtime デプロイメントを設定中...\n")

# Runtime 用の実行ロールを作成
print("🔐 実行ロールを作成中...")
execution_role_arn = create_agentcore_runtime_execution_role()
print(f"✅ 実行ロールを作成しました: {execution_role_arn}")

# Runtime ツールキットを初期化
agentcore_runtime = Runtime()

# デプロイメントを設定
print("\n📋 デプロイメントパラメータを設定中...")
response = agentcore_runtime.configure(
    entrypoint="strands_agent.py",
    execution_role=execution_role_arn,
    auto_create_ecr=True,  # ECR リポジトリを自動作成
    requirements_file="requirements.txt",
    region=REGION,
    agent_name="customer_support_elasticsearch_agent",
    non_interactive=True,
    memory_mode="NO_MEMORY",  # この例では AgentCore Memory を使用しません
    # Cognito JWT 認可を設定
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [client_id],
            "discoveryUrl": cognito_discovery_url
        }
    },
    # Authorization ヘッダーをエージェントに転送することを許可
    request_header_configuration={
        "requestHeaderAllowlist": [
            "Authorization"  # OAuth トークン伝播に必要
        ]
    },
)

print("\n✅ 設定が正常に完了しました")
print("\n📄 生成されたファイル:")
print("   - Dockerfile")
print("   - .bedrock_agentcore.yaml")

### ステップ 3: エージェントを Runtime に起動

ここからがエキサイティングな部分です。エージェントを本番環境に起動します！このステップでは以下を行います：

1. **Docker イメージのビルド**: エージェントコードと依存関係を含むコンテナを作成
2. **ECR へのプッシュ**: イメージを Amazon Elastic Container Registry にアップロード
3. **Runtime の作成**: AgentCore Runtime インフラストラクチャをプロビジョニング
4. **ネットワーキングの設定**: セキュアな HTTPS エンドポイントをセットアップ
5. **コンテナのデプロイ**: マネージド環境でエージェントを実行

> ⏱️ **デプロイメント時間**: 初回デプロイメントには通常最大 10 分かかることがあります。ツールキットは、コンテナをビルドするための AWS CodeBuild プロジェクトを作成します。

> 💡 **既存の Runtime の更新**: 同じ名前のエージェントが既に存在する場合は、`auto_update_on_conflict=True` を使用して新しいものを作成する代わりに更新します。

In [ ]:
print("🚀 エージェントを AgentCore Runtime に起動中...\n")
print("このプロセスでは以下を行います:")
print("  1. Docker コンテナをビルド")
print("  2. Amazon ECR にプッシュ")
print("  3. AgentCore Runtime にデプロイ")
print("  4. 認証とネットワーキングを設定\n")
print("⏳ 数分かかる場合があります...\n")

try:
    # エージェントを起動
    launch_result = agentcore_runtime.launch()
    
    print("\n✅ 起動が正常に開始されました！")
    print(f"   エージェント ARN: {launch_result.agent_arn}")
    print(f"   エージェント ID: {launch_result.agent_id}")
    print(f"   ECR URI: {launch_result.ecr_uri}")
    
except Exception as e:
    print(f"\n❌ 起動に失敗しました: {e}")
    print("\n💡 ヒント: エージェント名が既に存在する場合は、更新できます:")
    print("   launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)")
    raise

## 8. デプロイメントのテスト

おめでとうございます！エージェントがデプロイされ、実行されています。様々なカスタマーサポートシナリオでテストしましょう。

### テスト戦略

3 つの主要なシナリオをテストします：
1. **製品トラブルシューティング**: 特定の製品クエリによる Elasticsearch 取得のテスト
2. **一般情報**: ナレッジベース検索が必要なクエリをエージェントがどのように処理するかのテスト
3. **セッション継続性**: エージェントのセッション処理の検証

### テスト用の認証

まず、新しい OAuth トークンが必要です（トークンは一定期間後に期限切れになります）：

In [ ]:
print("🔑 テスト用の新しい OAuth トークンを取得中...\n")

try:
    token_response = utils.get_token(user_pool_id, client_id, client_secret, scope_string, REGION)
    test_token = token_response["access_token"]
    
    print("✅ トークンを正常に取得しました")
    print(f"   トークン（省略）: {test_token[:50]}...")
except Exception as e:
    print(f"❌ トークン取得エラー: {e}")
    raise

### テストシナリオ 1: 製品トラブルシューティング

特定の製品のトラブルシューティングについて尋ねる顧客でエージェントをテストしましょう：

In [ ]:
print("🧪 テスト 1: 製品トラブルシューティング\n")
print("=" * 70)

user_query = "How can I troubleshoot my LAPTOP ULTRA 15.6?"
print(f"👤 顧客: {user_query}\n")

try:
    response = agentcore_runtime.invoke(
        {"prompt": user_query},
        bearer_token=test_token,
    )
    
    print(f"🤖 エージェントの応答:")
    print("-" * 70)
    print(response["response"])
    print("=" * 70)
    print("\n✅ テスト 1 が正常に完了しました\n")
    
except Exception as e:
    print(f"❌ テストに失敗しました: {e}\n")

### テストシナリオ 2: 一般的な製品情報

製品サポートに関する別のタイプのクエリでテストしましょう：

In [ ]:
print("🧪 テスト 2: 一般的な製品情報\n")
print("=" * 70)

user_query = "What number can I call for health related questions on my Smart Watch Series X?"
print(f"👤 顧客: {user_query}\n")

try:
    response = agentcore_runtime.invoke(
        {"prompt": user_query},
        bearer_token=test_token,
    )
    
    print(f"🤖 エージェントの応答:")
    print("-" * 70)
    print(response["response"])
    print("=" * 70)
    print("\n✅ テスト 2 が正常に完了しました\n")
    
except Exception as e:
    print(f"❌ テストに失敗しました: {e}\n")

### テストシナリオ 3: セッション管理

フォローアップの質問をしてセッション継続性をテストしましょう：

In [ ]:
print("🧪 テスト 3: セッション管理\n")
print("=" * 70)

# 継続性のためにセッション ID を作成
session_id = f"test-session-{uuid.uuid4()}"

# 最初のクエリ
user_query_1 = "What information do you have for Smart Watch Series X?"
print(f"👤 顧客（メッセージ 1）: {user_query_1}\n")

response_1 = agentcore_runtime.invoke(
    {"prompt": user_query_1},
    bearer_token=test_token,
    session_id=session_id
)

print(f"🤖 エージェント: {response_1['response'][:200]}...\n")

# フォローアップクエリ
user_query_2 = "Can you tell me more about the specific product I asked about?"
print(f"👤 顧客（メッセージ 2）: {user_query_2}\n")

response_2 = agentcore_runtime.invoke(
    {"prompt": user_query_2},
    bearer_token=test_token,
    session_id=session_id
)

print(f"🤖 エージェント:")
print("-" * 70)
print(response_2['response'])
print("=" * 70)
print("\n✅ テスト 3 が正常に完了しました\n")

## 主要概念のまとめ

このチュートリアルを完了おめでとうございます！学んだ主要概念を振り返りましょう：

### 1. AgentCore Gateway アーキテクチャ
- **MCP 変換**: Lambda 関数を Model Context Protocol（MCP）ツールに変換する方法
- **デュアル認証**: インバウンド（OAuth）とアウトバウンド（IAM）の両方の認証の実装
- **マネージドインフラストラクチャ**: フルマネージド Gateway サービスを活用して運用オーバーヘッドを排除

### 2. 認証とセキュリティ
- **Cognito 統合**: Amazon Cognito で OAuth 2.0 認証をセットアップ
- **トークン伝播**: Runtime から Gateway への JWT トークンの転送
- **IAM ロール**: Gateway と Runtime の最小権限アクセスの設定
- **多層防御**: クライアントからデータストアまでの複数のセキュリティレイヤー

### 3. Elasticsearch 統合
- **RAG パターン**: Elasticsearch で Retrieval-Augmented Generation を実装
- **データブリッジとしての Lambda**: Lambda を使用して Gateway を Elasticsearch にセキュアに接続
- **セマンティック検索**: 関連ドキュメント取得のためのマルチマッチクエリの実行

### 4. AgentCore Runtime デプロイメント
- **コンテナ化**: 移植性のためにエージェントを Docker コンテナとしてパッケージング
- **スターターツールキット**: 自動化ツールを使用してデプロイメントプロセスを簡素化
- **本番準備**: スケーラブルでマネージドなエージェントサービスのデプロイ
- **ヘッダー設定**: エージェントへの認証およびカスタムヘッダーの転送

### 5. MCP プロトコル
- **標準インターフェース**: 一貫したツール統合のための MCP の使用
- **ツール検出**: `ListTools` 操作による動的なツールロード
- **ツール呼び出し**: `InvokeTools` 操作によるツールの実行
- **クライアントライブラリ**: シームレスな統合のための MCP クライアント SDK の活用

### アーキテクチャのメリット

このアーキテクチャは、いくつかの本番環境での利点を提供します：

✅ **スケーラビリティ**: すべてのコンポーネント（Gateway、Runtime、Lambda）が自動的にスケール

✅ **セキュリティ**: 複数の認証レイヤーがすべてのアクセスポイントを保護

✅ **保守性**: 関心の分離により、更新とデバッグが容易

✅ **柔軟性**: 新しいツール、データソース、エージェントの追加が容易

✅ **コスト効率**: サーバーレスコンポーネントにより実際の使用量のみに課金

## クリーンアップ（オプション）

不要な AWS 料金が発生しないように、このチュートリアルで作成したリソースを削除できます。このセクションでは、以下のクリーンアップをガイドします：

1. **AgentCore Runtime エージェント** - デプロイされたエージェントコンテナ
2. **AgentCore Gateway** - MCP Gateway とターゲット
3. **Lambda 関数** - Elasticsearch クエリ関数
4. **Cognito リソース** - ユーザープールとアプリクライアント
5. **IAM ロール** - サービス用に作成された実行ロール
6. **ECR リポジトリ** - コンテナイメージリポジトリ
7. **SSM パラメータ** - 保存された設定値

> ⚠️ **警告**: この操作は元に戻せません。続行する前に、これらのリソースを削除することを確認してください。

### クリーンアップ手順

In [ ]:
# ⚠️ 警告: すべての作成されたリソースを削除する場合のみこのセルを実行してください

print("🧹 クリーンアッププロセスを開始中...\n")
print("=" * 70)

cleanup_errors = []

# 1. AgentCore Runtime を削除
print("\n1️⃣ AgentCore Runtime を削除中...")
try:
    if 'launch_result' in locals() and hasattr(launch_result, 'agent_id'):
        runtime_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
        runtime_client.delete_agent_runtime(agentRuntimeId=launch_result.agent_id)
        print(f"   ✅ Runtime を削除しました: {launch_result.agent_id}")
    else:
        print("   ⏭️  削除する Runtime がありません")
except Exception as e:
    print(f"   ❌ エラー: {e}")
    cleanup_errors.append(f"Runtime 削除: {e}")

# 2. Gateway ターゲットを削除
print("\n2️⃣ Gateway ターゲットを削除中...")
try:
    if 'target_id' in locals() and 'gateway_id' in locals():
        gateway_client.delete_gateway_target(
            gatewayIdentifier=gateway_id,
            targetId=target_id
        )
        print(f"   ✅ ターゲットを削除しました: {target_id}")
    else:
        print("   ⏭️  削除する Gateway ターゲットがありません")
except Exception as e:
    print(f"   ❌ エラー: {e}")
    cleanup_errors.append(f"Gateway ターゲット削除: {e}")

# 3. Gateway を削除
time.sleep(10)
print("\n3️⃣ AgentCore Gateway を削除中...")
try:
    if 'gateway_id' in locals():
        gateway_client.delete_gateway(gatewayIdentifier=gateway_id)
        print(f"   ✅ Gateway を削除しました: {gateway_id}")
    else:
        print("   ⏭️  削除する Gateway がありません")
except Exception as e:
    print(f"   ❌ エラー: {e}")
    cleanup_errors.append(f"Gateway 削除: {e}")

# 4. Lambda 関数を削除
print("\n4️⃣ Lambda 関数を削除中...")
try:
    if 'lambda_function_arn' in locals():
        lambda_client = boto3.client('lambda', region_name=REGION)
        function_name = lambda_function_arn.split(':')[-1]
        lambda_client.delete_function(FunctionName=function_name)
        print(f"   ✅ Lambda を削除しました: {function_name}")
    else:
        print("   ⏭️  削除する Lambda 関数がありません")
except Exception as e:
    print(f"   ❌ エラー: {e}")
    cleanup_errors.append(f"Lambda 削除: {e}")

# 5. Cognito ユーザープールを削除
print("\n5️⃣ Cognito ユーザープールを削除中…")
try:
    if 'user_pool_id' in locals():
        # 最初にドメインが存在する場合は削除
        try:
            response = cognito.describe_user_pool(UserPoolId=user_pool_id)
            domain = response.get('UserPool', {}).get('Domain')
            if domain:
                cognito.delete_user_pool_domain(
                    Domain=domain,
                    UserPoolId=user_pool_id
                )
                print(f"   ✅ ユーザープールドメインを削除しました: {domain}")
        except cognito.exceptions.ResourceNotFoundException:
            print("   ⏭️  ドメインが見つかりません")
        except Exception as domain_error:
            print(f"   ⚠️  ドメイン削除の警告: {domain_error}")
        
        # ユーザープールを削除
        cognito.delete_user_pool(UserPoolId=user_pool_id)
        print(f"   ✅ ユーザープールを削除しました: {user_pool_id}")
    else:
        print("   ⏭️  削除するユーザープールがありません")
except Exception as e:
    print(f"   ❌ エラー: {e}")
    cleanup_errors.append(f"Cognito 削除: {e}")

# 6. ECR リポジトリを削除
print("\n6️⃣ ECR リポジトリを削除中...")
try:
    if 'launch_result' in locals() and hasattr(launch_result, 'ecr_uri'):
        ecr_client = boto3.client('ecr', region_name=REGION)
        repo_name = launch_result.ecr_uri.split('/')[1].split(':')[0]
        ecr_client.delete_repository(repositoryName=repo_name, force=True)
        print(f"   ✅ ECR リポジトリを削除しました: {repo_name}")
    else:
        print("   ⏭️  削除する ECR リポジトリがありません")
except Exception as e:
    print(f"   ❌ エラー: {e}")
    cleanup_errors.append(f"ECR 削除: {e}")

# 7. SSM パラメータを削除
print("\n7️⃣ SSM パラメータを削除中...")
try:
    ssm_client = boto3.client('ssm', region_name=REGION)
    ssm_client.delete_parameter(Name="/app/customer_support_elastic/agentcore/gateway_id")
    print("   ✅ SSM パラメータを削除しました")
except Exception as e:
    print(f"   ⏭️  削除する SSM パラメータがありません（または既に削除済み）")

# サマリー
print("\n" + "=" * 70)
if cleanup_errors:
    print("⚠️  クリーンアップがエラーで完了しました:")
    for error in cleanup_errors:
        print(f"   - {error}")
else:
    print("✅ クリーンアップが正常に完了しました！")
print("=" * 70)

print("\n💡 注意: IAM ロールは AWS IAM の結果整合性により削除に遅延が発生する場合があります。")
print("   必要に応じて IAM コンソールから手動で削除できます。")